# Experiment 4: Air Track II---Newton’s Second Law
### Objectives
- To learn to take good measurements
- To learn how to estimate and propagate errors
- To learn how to take measurements to verify a theory

### Equipment
- One Airtrack, Blower, and Cart
- One accessory kit containing: one pulley, one mass hanger, and masses for cart and hanger.
- Two (2) photogates
- One 30 cm ruler
- One vernier caliper
- One two-meter stick
- One scale

### Safety
- Be careful placing the carts on the track. Do not damage the track.
- Please place paper underneath the carts when they are resting on the track without the air turned on. **Do not slide the carts on the track without the air turned on.**

# Introduction
In 1686, Isaac Newton published his three laws of motion using information he gained from experiments performed by both Galileo Galilei and Johannes Kepler:
1. If there is no net force, there is no acceleration
2. If there is a net force, then *F = ma*
3. If object A places a force on object B, then object B places an equal but opposite force on object A.

Today you will be verifying Newton’s second law by using a known force to accelerate an airtrack glider, measuring that acceleration, and comparing it to the predicted acceleration.

# Theory
### Kinematics
First and foremost, you will need to compute the acceleration of the aitrack glider. The information you will have, however, is velocity computed by a photogate. The photogate measures the velocity of the glider as it passes through by timing how long the infrared LED is blocked by the object (sometimes called a “flag”) passing through and uses the definition of velocity:  

\begin{equation}
v \equiv \frac{\Delta x}{\Delta t}
\tag{1}
\end{equation}

where $\Delta x$ is the length of the flag and $\Delta t$ is the time the LED was blocked.

To determine the acceleration of the glider, consider the definition of acceleration:  
\begin{equation}
a_{x} \equiv \frac{\Delta v_{x}}{\Delta t}
\tag{2}
\end{equation}

To use this, you will need to know the velocity of the cart at two different times. Think carefully about how you might measure those times.

Keep in mind that you will need to determine the errors in your measurements and propagate those errors into your computed acceleration.

### Force
A proper prediction of the acceleration of the glider from Newton’s second law requires great care. Recall that the force of gravity on an object is given by $F = mg$ where $g \equiv 9.81 \pm 0.005 m/s^2$ so that $ \delta g = 0.005 m/s^2$. You should find that the acceleration of the system is some fraction of g involving the masses of both the glider and the hanging mass.

### Uncertainty Analysis
You will need to compute the uncertainty in both your predicted and your measured accelerations. The uncertainty in your predicted acceleration will depend on the uncertainty in “g” as well as the uncertainties in the masses. The uncertainty in your “measured” acceleration will depend on the uncertainties in your measured velocities and times.

The Capstone software can measure time to within $ \delta t = 0.1 ms$.

Your Lab Manual and the previous labs can guide you in estimating the uncertainties in your measurements and propagating those uncertainties into your computed accelerations. The uncertainty in the masses of the glider and hanging mass will be determined by the accuracy of the scale. For your velocities, it might be easier to use the standard deviation of a large number of measurements as your measurement uncertainty. The uncertainty in your acceleration, however, will have to be computed using the three equations in your Lab Manual.

In order to minimize random errors, it is extremely important that each of your measurements be performed several times.

# Experimental Procedure

### Setting up the Air Track
Ensure that the pulley is securely inserted into the top hole in the bracket at the far end of the track and that it spins freely. Make sure that the blower is functioning properly (you should be able to feel the air streaming out of the pin-holes). Make sure that the hanging mass will not strike the table as it falls.

You will need to level the airtrack so that the only force comes from the hanging mass. Look back to the second lab for instructions if you’ve forgotten how to do this.

### Setting up a Photogate
1. Turn on the PASCO 850 Interface and start the PASCO Capstone software.
2. Plug a photogate into a Digital input.
3. Click the “Hardware Setup” tab in the left “Tools” palette, left-click the jack on the diagram where you inserted the plug, and select “Photogate” from the drop-down menu.
4. You should see a tab labeled “Timer Setup.” Open that tab and set up a pre-configured timer:
    1. Select the photogate you just installed.
    2. You will be using this photogate with a single flag.
    3. The computer need only keep track of the speed through the gate.
    4. A text box requesting the length of the flag in meters will appear. Measure the flag as best you can and enter that information in the box.
    5. Give this sensor a name such as “Photogate 1” or similar.

Repeat steps 2—4e for the second photogate.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys; sys.path.insert(0, '..')
from P201_Functions import *

# Flag length measurement (m) and its uncertainty
flag_len = 0.1
delta_x = 0.005

# uncertainty in measured times
delta_t = 0.001

# Acceleration Due to Gravity
g = 9.81
dg = 0.005

In [2]:
def massCalc(mass2, dmass2, mass1, dmass1):
    """
    Calculate the the theoretical force of tension and predicted acceleraton for a given cart mass and hanging mass
    :param mass2: cart mass (kg)
    :param dmass2: cart mass uncertainty (kg)
    :param mass1: hanging mass (kg)
    :param dmass1: hanging mass uncertainty (kg)
    :return: tuple containing theoretical force of tension, uncertainty in FT, predicted acceleration, uncertainty in ap
    """
    
    # Theoretical Force of Tension (N)
    theo_FT = mass1*mass2/(mass1+mass2)*g

    # calculate uncertainty in T
    dTdmass1 = mass2*g*(mass2/(mass1+mass2)**2)
    dTdmass2 = mass1*g*(mass1/(mass1+mass2)**2)
    dTdg = mass1*mass2/(mass1+mass2)
    dtheo_FT = dTdmass1*dmass1 + dTdmass2*dmass2 + dTdg*dg
    #print (theo_FT,dtheo_FT)

    # Predicted Acceleration (m/s^2) and its uncertainty
    a_p = mass1/(mass1+mass2)*g

    # calculate uncertainty in a
    dadmass1 = g*(mass2/(mass1+mass2)**2)
    dadmass2 = g*(mass1/(mass1+mass2)**2)
    dadg = mass1/(mass1+mass2)
    da_p = dadmass1*dmass1 + dadmass2*dmass2 + dadg*dg
    #print (a_p,da_p)
    
    return theo_FT, dtheo_FT, a_p, da_p

def runCalc(df):
    """
    calculates the uncertanties and acceleration for a cart track run
    :param df: a pandas dataframe containing v1,t1,v2,t2 data
    :return: tuple containing pandas daraframe containing [v1,t1,v2,t2,a,𝛿v,𝛿am], average measured acceleration, and average acceleration uncertainty
    """
    # Calculates the change in velocity and time 
    Delta_v = df['v2 (m/s)'] - df['v1 (m/s)']
    Delta_t = df['t2 (s)'] - df['t1 (s)']

    # Calculates acceleration
    df['a (m/s^2)'] = Delta_v/Delta_t

    # Calculates the uncertainty of the velocity based on Equation 1
    uncertainty_v = Delta_v*(delta_x/flag_len + delta_t/Delta_t)
    df["𝛿v (m/s^2)"] =  uncertainty_v

    # Finds the uncertainty of acceleration and adds it to the dataframe
    unc_am = df['a (m/s^2)']*((uncertainty_v/Delta_v) + (delta_t/Delta_t))
    df["𝛿am (m/s^2)"] = unc_am

    # Finds the average measured acceleration
    av_am = sum(df['a (m/s^2)'])/len(df['a (m/s^2)'])
    dav_am = sum(df["𝛿am (m/s^2)"])/len(df["𝛿am (m/s^2)"])/np.sqrt(len(df["𝛿am (m/s^2)"]))
    
    return df, av_am, dav_am

def printData(m2,dm2,m1,dm1,a_p,da_p,theo_FT,dtheo_FT,av_am,dav_am):
    """
    prints out nicely formatted run data.
    """
    # prints out the cart data
    print("Data for: ")
    print("m_cart = %0.5f kg              𝛿m_cart = %0.6f" % (m2, dm2))
    print("m_hang = %0.5f kg              𝛿m_hang = %0.6f" % (m1, dm1))
    print("Predicted a_p = %0.3f m/s^2      𝛿a_p = %0.4f" % (a_p, da_p))
    print("Theoretical F_T = %0.5f N        𝛿F_T = %0.4f"      % (theo_FT,dtheo_FT))
    print()
    print('-'*70)

    # Prints information about the file
    #df.info()
    print(df)
    print()
    # Print statements for uncertainty + average
    print("Average am = %0.5f  +/- %0.5f m/s^2" % (av_am,dav_am))
    print('-'*70)
    print()


When you adjust the height of the photogate, make sure that it is triggered only by the flag. The string should not trigger the photogate. You can tell when the photogate is triggered by looking for the red LED to light when the infrared LED is blocked. Ensure that the red LED only lights when the proper portion of the cart passes through.

Something to think about when choosing the positions for your photogates: Where should the mass hanger be while the cart goes through a photogate?

### Data Collection
<u>**To release the cart: Hold the cart in position and turn on the blower. The cart must be released when the blower is running at full speed.**</u>

Capstone will display the photogate data any way you like. The best tool for displaying the data from a photogate, however, might be a table that plots the time that the photogate was triggered in one column and the velocity through that photogate in the other. You will need one table for each photogate.

Your goal is to confirm or deny Newton’s Second Law: $F = ma$. Notice that, if you had data points for the force (tension), *F*, as a function of the acceleration, *a*, you could create a graph. If that graph had the right shape, you might say that you had confirmed Newton’s Second Law. How should you plot this graph so that the slope has meaning?

You will need to plot a few different points of tension versus acceleration for each cart mass---*at least five* points, but the more data you have, the better your graph will be. Therefore, pick a cart mass and measure its acceleration for each of five different hanging masses. Once you’ve done that, change the cart mass and repeat the experiment. Please make sure that your cart does not bounce back through the second photogate after striking the bumper at the end of the track.

Your final result should be a graph of the force on the glider vs. the acceleration of the glider taken at a number of different masses for the hanging mass and the glider mass.

**Remember: Time is short---taking data is your first priority.** However, if the data you are taking is poor, then you might as well not be taking any data at all. Always dedicate one team member to compute the results of your experiments (in this case, they should be computing your measured accelerations) to see if your data matches your theoretical prediction. If your data is *not* matching your predictions, you should stop *and* diagnose the problem (check both your experiment and your prediction) before continuing.

You <u>must</u> take data with at least <u>two different cart masses</u>. If there is time, you can create a third graph with a third cart mass if you like. But you might make a preliminary plot of the data you have and try and fill in any gaps in that data, first.

In [3]:
# Raw Data 
# Cart Mass 1

# Mass of the cart (kg) and its uncertainty
m2  = 0.20130
dm2 = 0.000005

#### HANGING MASS #1 DATA HERE!!!!!!!!!!!!!! ####

# Mass of hanging mass (kg) and its uncertainty
m1  = 0.01285
dm1 = 0.000005

theo_FT, dtheo_FT, a_p, da_p = massCalc(m2,dm2,m1,dm1)

df = pd.DataFrame()

#### ENTER RUN DATA HERE: [Run 1, Run2, Run3, Run4, Run5]

df['v1 (m/s)'] = [0.32,0.31,0.32,0.32,0.32]
df['t1 (s)'] = [4.776,3.777,4.161,2.296,3.899]
df['v2 (m/s)'] = [0.86,0.87,0.86,0.86,0.86]
df['t2 (s)'] = [5.709,4.704,5.093,3.231,4.832]
#print(df)

df, av_am, dav_am = runCalc(df)

printData(m2,dm2,m1,dm1,a_p,da_p,theo_FT,dtheo_FT,av_am,dav_am)

############################################




Data for: 
m_cart = 0.20130 kg              𝛿m_cart = 0.000005
m_hang = 0.01285 kg              𝛿m_hang = 0.000005
Predicted a_p = 0.589 m/s^2      𝛿a_p = 0.0005
Theoretical F_T = 0.11849 N        𝛿F_T = 0.0001

----------------------------------------------------------------------
   v1 (m/s)  t1 (s)  v2 (m/s)  t2 (s)  a (m/s^2)  𝛿v (m/s^2)  𝛿am (m/s^2)
0      0.32   4.776      0.86   5.709   0.578778    0.027579     0.030180
1      0.31   3.777      0.87   4.704   0.604099    0.028604     0.031508
2      0.32   4.161      0.86   5.093   0.579399    0.027579     0.030213
3      0.32   2.296      0.86   3.231   0.577540    0.027578     0.030112
4      0.32   3.899      0.86   4.832   0.578778    0.027579     0.030180

Average am = 0.58372  +/- 0.01361 m/s^2
----------------------------------------------------------------------



***

In [4]:
# Raw Data 
# Cart Mass 2

# Mass of the cart (kg) and its uncertainty
m2  = 0.20130
dm2 = 0.000005

#### HANGING MASS #1 DATA HERE!!!!!!!!!!!!!! ####

# Mass of hanging mass (kg) and its uncertainty
m1  = 0.01285
dm1 = 0.000005

theo_FT, dtheo_FT, a_p, da_p = massCalc(m2,dm2,m1,dm1)

df = pd.DataFrame()

#### ENTER RUN DATA HERE: [Run 1, Run2, Run3, Run4, Run5]

df['v1 (m/s)'] = [0.32,0.31,0.32,0.32,0.32]
df['t1 (s)'] = [4.776,3.777,4.161,2.296,3.899]
df['v2 (m/s)'] = [0.86,0.87,0.86,0.86,0.86]
df['t2 (s)'] = [5.709,4.704,5.093,3.231,4.832]
#print(df)

df, av_am, dav_am = runCalc(df)

printData(m2,dm2,m1,dm1,a_p,da_p,theo_FT,dtheo_FT,av_am,dav_am)

############################################

Data for: 
m_cart = 0.20130 kg              𝛿m_cart = 0.000005
m_hang = 0.01285 kg              𝛿m_hang = 0.000005
Predicted a_p = 0.589 m/s^2      𝛿a_p = 0.0005
Theoretical F_T = 0.11849 N        𝛿F_T = 0.0001

----------------------------------------------------------------------
   v1 (m/s)  t1 (s)  v2 (m/s)  t2 (s)  a (m/s^2)  𝛿v (m/s^2)  𝛿am (m/s^2)
0      0.32   4.776      0.86   5.709   0.578778    0.027579     0.030180
1      0.31   3.777      0.87   4.704   0.604099    0.028604     0.031508
2      0.32   4.161      0.86   5.093   0.579399    0.027579     0.030213
3      0.32   2.296      0.86   3.231   0.577540    0.027578     0.030112
4      0.32   3.899      0.86   4.832   0.578778    0.027579     0.030180

Average am = 0.58372  +/- 0.01361 m/s^2
----------------------------------------------------------------------



***

In [5]:
# Raw Data 
# Cart Mass 3

# Mass of the cart (kg) and its uncertainty
m2  = 0.20130
dm2 = 0.000005

#### HANGING MASS #1 DATA HERE!!!!!!!!!!!!!! ####

# Mass of hanging mass (kg) and its uncertainty
m1  = 0.01285
dm1 = 0.000005

theo_FT, dtheo_FT, a_p, da_p = massCalc(m2,dm2,m1,dm1)

df = pd.DataFrame()

#### ENTER RUN DATA HERE: [Run 1, Run2, Run3, Run4, Run5]

df['v1 (m/s)'] = [0.32,0.31,0.32,0.32,0.32]
df['t1 (s)'] = [4.776,3.777,4.161,2.296,3.899]
df['v2 (m/s)'] = [0.86,0.87,0.86,0.86,0.86]
df['t2 (s)'] = [5.709,4.704,5.093,3.231,4.832]
#print(df)

df, av_am, dav_am = runCalc(df)

printData(m2,dm2,m1,dm1,a_p,da_p,theo_FT,dtheo_FT,av_am,dav_am)

############################################

Data for: 
m_cart = 0.20130 kg              𝛿m_cart = 0.000005
m_hang = 0.01285 kg              𝛿m_hang = 0.000005
Predicted a_p = 0.589 m/s^2      𝛿a_p = 0.0005
Theoretical F_T = 0.11849 N        𝛿F_T = 0.0001

----------------------------------------------------------------------
   v1 (m/s)  t1 (s)  v2 (m/s)  t2 (s)  a (m/s^2)  𝛿v (m/s^2)  𝛿am (m/s^2)
0      0.32   4.776      0.86   5.709   0.578778    0.027579     0.030180
1      0.31   3.777      0.87   4.704   0.604099    0.028604     0.031508
2      0.32   4.161      0.86   5.093   0.579399    0.027579     0.030213
3      0.32   2.296      0.86   3.231   0.577540    0.027578     0.030112
4      0.32   3.899      0.86   4.832   0.578778    0.027579     0.030180

Average am = 0.58372  +/- 0.01361 m/s^2
----------------------------------------------------------------------



***

# Your Report
### Experimental Methods
You will need to write a few paragraphs to carefully describe how you set up and executed your
experiment. You must include the following items:
- Explain your procedure to ensure that the acceleration of the cart was caused only by the gravitational force due to the hanging mass.
- What did you use for your “flag” for the photogate?
- How did you take your data to reduce random errors?

Write your procedure section so that someone else could perform the experiment by just reading it (but do not write instructions). Talk about any problems you encountered and how you handled them.

### Results and Conclusion
This will be your longest section. Remember that your goal is to compare your theoretical prediction with your experimental results. Use good paragraph structure to link the results together:
- How do the slopes of your Tension vs. Acceleration graphs compare to the mass of the cart? Use the % error formula and discuss.
- Did you have any random errors? Any systematic errors? How do you know?
- Does your data support or refute Newton’s Second Law? *Why?*
- How might you reduce your uncertainties if you were to run the experiment again?


### Questions
1. Calculate the acceleration of a 0.5 kg cart with a 1.0 kg hanging mass.
2. Calculate the acceleration of a 1.0 kg cart with a 0.5 kg hanging mass.
3. Calculate the uncertainty of the acceleration for each of these. Which is lower?
4. Does the photogate measure the true instantaneous velocity of the cart? *Why or why not?*

### Appendix
*Include your signed raw data sheet(s)*  
*Include a sample calculation for every computation you need to make.*